In [206]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request as u
output = "data.csv" 
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
import folium
import plotly.express as ex
import plotly.graph_objects as go
from folium import plugins
import json
from plotly.subplots import make_subplots
import seaborn as sb
import chart_studio.plotly as py
import plotly
f = u.urlopen('https://www.mygov.in/corona-data/covid19-statewise-status')
myfile = f.read().decode().split('div class="field field-name-field-select-state field-type-list-text field-label-above"><div class="field-label">State Name:&nbsp;</div><div class="field-items"><div class="field-item even">')[1:]
states = list()
for a in myfile :
    states.append(a.split('<')[0])
data = dict()
for a, b in zip(states, myfile) :
    temp = list()
    temp.append(b.split('Total Confirmed:&nbsp;</div><div class="field-items"><div class="field-item even">')[1].split('<')[0])
    temp.append(b.split('Cured/ Discharged/ Migrated:&nbsp;</div><div class="field-items"><div class="field-item even">')[1].split('<')[0])
    temp.append(b.split('Death:&nbsp;</div><div class="field-items"><div class="field-item even">')[1].split('<')[0])
    data.update({a : temp})
st_data = pd.DataFrame(data).T
st_data.columns = ['Total Confirmed Cases', 'Total Cured/Discharged/Migrated', 'Total Deaths Recorded']
coor = pd.read_excel('Indian Coordinates.xlsx', index_col=0)
cor_i = list( a.strip() for a in coor.index)
coor.index = cor_i
st_data = pd.merge( coor ,st_data, right_index=True, left_index=True)
st_data = st_data.astype(float)
st_data["% of total"] = st_data['Total Confirmed Cases'] / st_data['Total Confirmed Cases'].sum()
wg = st_data["% of total"]
m = folium.Map(location=[22,80], zoom_start=5.1 , tiles='Stamen Toner' )
states = st_data.index
for lat, long, w , st in zip(st_data['Latitude'], st_data['Longitude'], wg, states  ) :
    folium.CircleMarker(location = [lat,long] , radius = w*300 , popup=[st , st_data.loc[st, 'Total Confirmed Cases']], color = 'red', fill_color = 'red').add_to(m)
m

In [207]:
%matplotlib inline
df = pd.read_csv(url , index_col = 'date' , parse_dates=True)
df = df[(df['location'] == 'India') | (df['location'] == 'United States') | (df['location'] == 'Italy') | (df['location'] == 'South Korea') ]
df.drop(['iso_code', 'continent', 'total_cases_per_million', 'tests_units', 
       'new_cases_per_million', 'total_deaths_per_million', 
       'new_deaths_per_million','total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'gdp_per_capita', 'extreme_poverty',
        'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 
       'life_expectancy'] , axis = 1 , inplace = True)
pd.set_option("display.max_rows" , 30)
df.fillna(method = 'ffill' , axis = 0 , inplace=True)
df.dropna(inplace = True)
df['cases per test done'] = df['total_cases']/df['total_tests']
df["Rate of Increase in cases"] = (df['total_cases'].pct_change())*100
df["Rate of Increase in testing"] = df['total_tests'].pct_change()*100
df['Rate of Increase in Deaths'] = df["total_deaths"].pct_change()*100
df["Rate of Increase in cases per test"] = df["cases per test done"].pct_change()*100
df["New Cases per test done in a day"] = df["new_cases"]/df['new_tests']
df["Rate of Increase in New Cases per test done in a day"] = df["New Cases per test done in a day"].pct_change()*100
df.dropna(inplace = True)
df.iloc[0:, 1:] = df.iloc[0:, 1:].astype(float)
df = df['2020-03-01':]
cou_list = df.groupby('location').sum().index.values.tolist()
colors = dict(zip(cou_list, sb.color_palette("inferno_r", len(cou_list)).as_hex()))
trace_list = []
for cou in cou_list:
    trace = go.Scatter(
        y=df[df['location']==cou]['total_cases'].tolist(),
        x=df[df['location']==cou].index,
        mode='lines+markers',
        name=cou,
        line = dict(
                    color = colors[cou],
                    width = 1.5,
                    )
        )
    trace_list.append(trace)
    
    
layout = go.Layout(
    xaxis=dict(title='Date', zeroline=False, rangeslider=dict(visible=True)),
    yaxis=dict(title='Number of Total Cases', zeroline=False),

    title='Linear Trend of Total Coronavius Cases in the world',
    showlegend=True,
    )

fig = go.Figure(data=trace_list, layout=layout)
fig

trace_list = []
for cou in cou_list:
    trace = go.Scatter(
        y=df[df['location']==cou]['total_tests'].tolist(),
        x=df[df['location']==cou].index,
        mode='lines+markers',
        name=cou,
        line = dict(
                    color = colors[cou],
                    width = 1.5,
                    )
        )
    trace_list.append(trace)
    
    
layout = go.Layout(
    xaxis=dict(title='Date', zeroline=False, rangeslider=dict(visible=True)),
    yaxis=dict(title='Number of Total Tests Done', zeroline=False),

    title='Linear Trend of Total Coronavius Testing Done in the world',
    showlegend=True,
    )

fig1 = go.Figure(data=trace_list, layout=layout)

trace_list = []
for cou in cou_list:
    trace = go.Scatter(
        y=df[df['location']==cou]['New Cases per test done in a day'].tolist(),
        x=df[df['location']==cou].index,
        mode='lines',
        name=cou,
        line = dict(
                    color = colors[cou],
                    width = 1.5,
                    )
        )
    trace_list.append(trace)
    
    
layout = go.Layout(
    xaxis=dict(title='Date', zeroline=False, rangeslider=dict(visible=True)),
    yaxis=dict(title='New Cases per test done in a day', zeroline=False),

    title='Trend of New Cases per test done in a day',
    showlegend=True,
    )

fig2 = go.Figure(data=trace_list, layout=layout)


fig.show()
fig1.show()
fig2.show()

# Times Seires Prediction using fbprophet

In [208]:
from fbprophet import Prophet
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
df_new = pd.read_csv(url , index_col = 'date' , parse_dates=True)
df_new = df_new[df_new['location'] == 'India']
test = df_new[['total_cases']]['2020-04-15':]
test.dropna(inplace = True)
test.reset_index(inplace = True)
test.rename(columns = {'total_cases' : 'y' , 'date' : 'ds'} , inplace = True)
pr = Prophet(interval_width=0.01)
pr.fit(test)
fu = pr.make_future_dataframe(periods=10)
future = pr.predict(fu)
trace = go.Scatter(
        y=future['yhat'][-42:],
        x=future.ds[-42:],
        mode='lines+markers',
        name='India',
        )
    
    
layout = go.Layout(
    xaxis=dict(title='Date', zeroline=False, rangeslider=dict(visible=True)),
    yaxis=dict(title='Prediction of Number of Total Cases', zeroline=False),

    title='Linear Trend of Total Coronavius Cases in India for the past one month & prediction for the next ten days',
    showlegend=True,
    )

fig5 = go.Figure(data=trace, layout=layout)
fig5

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
